# Experiments on Tumour / Cell Line and the UCI Income datasets

In this notebook, we load the inferred pretrained models 

In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import seaborn as sns
import torch
from umap import UMAP
from tqdm import tqdm
import scipy
from scipy.stats import sem

from comp.metrics import knn_metric, silhouette_coeff, kbet

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import scipy

# 1. Download files

In [ ]:
models = ["comp", "cvae", "trvae", "vae", "vfae"]
metrics = ["silhouette", "mean-silhouette", "kbet", "mean-kbet"]

### Tumour / Cell Line files

In [ ]:
tumour_cl_z = {}
for model in models:
    umap_file = list(Path(f"celligner/{model}/run_01/umaps").rglob("*.parquet"))  # To change
    if len(umap_file) > 0:
        metadata_file = Path(f"celligner/metadata.tsv")
        umap = pd.read_parquet(umap_file[0])
        metadata = pd.read_csv(metadata_file, index_col=0, sep="\t")
        tumour_cl_z[model] = pd.concat([umap, metadata], axis=1)

### UCI files

In [ ]:
uci_z = {}
for model in models:
    umap_file = list(Path(f"uci/{model}/run_01/umaps").rglob("*.parquet")) # To change
    if len(umap_file) > 0:
        metadata_file = Path(f"uci/metadata.tsv")
        umap = pd.read_parquet(umap_file[0])
        metadata = pd.read_csv(metadata_file, index_col=0, sep="\t")
        uci_z[model] = pd.concat([umap, metadata], axis=1)

# 2. Compute metrics

## 2.1 Tumour / Cell Line metrics

In [ ]:
tumour_cl_metrics = pd.DataFrame(columns=metrics, index=models)

In [ ]:
def select_diseases(df, threshold=400):
    df2 = df.groupby("disease").agg({"subtype": "count", "type": 'nunique'})
    df3 = df2[(df2.subtype > 400) & (df2.type == 2)]
    return list(df3.index)

In [ ]:
selected_diseases = select_diseases(tumour_cl_z["comp"], threshold=400)  # 'comp' selection here is arbitrary; can be any other model.
print(f"No. of selected diseases: {len(selected_diseases)}")

#### kbet

In [ ]:
n_neighbours = 100
for model, data in tqdm(tumour_cl_z.items()):
    num_cl = sum((data.type == "CL").values)
    num_tumor = sum((data.type != "CL").values)
    freq_tumor = n_neighbours * num_tumor / (num_cl + num_tumor)
    freq_cl = n_neighbours - freq_tumor

    _, counts = knn_metric(features=data.iloc[:, :-3].values,
                           queries=[True] * data.shape[0], 
                           labels=data.disease.to_numpy(),
                           class_partition=(data.type == "CL").values,
                           n_neighbours=n_neighbours,
                           return_counts=True,
                           )
    expected_freq = np.where((data.type == "CL").values, freq_tumor, freq_cl)
    tumour_cl_metrics.loc[model, "kbet"] = kbet(counts[:, 1], expected_freq=expected_freq, n_neighbours=n_neighbours, significance=0.01)

#### mean-kbet

In [ ]:
n_neighbours = 100
kbet_all = []
for model, data_all in tqdm(tumour_cl_z.items()):
    metric_disease = []
    for disease in selected_diseases:
        data = data_all[data_all.disease == disease]
        num_cl = sum((data.type == "CL").values)
        num_tumor = sum((data.type != "CL").values)
        freq_tumor = n_neighbours * num_tumor / (num_cl + num_tumor)
        freq_cl = n_neighbours - freq_tumor

        _, counts = knn_metric(features=data.iloc[:, :-3].values,
                               queries=[True] * data.shape[0], 
                               labels=data.disease.to_numpy(),
                               class_partition=(data.type == "CL").values,
                               n_neighbours=n_neighbours,
                               return_counts=True,
                               )
        expected_freq = np.where((data.type == "CL").values, freq_tumor, freq_cl)
        metric_disease.append(kbet(counts[:, 1], expected_freq=expected_freq, n_neighbours=n_neighbours, significance=0.01))
    tumour_cl_metrics.loc[model, "mean-kbet"] = np.mean(metric_disease)

#### Silhouette

In [ ]:
for model, data in tqdm(tumour_cl_z.items()):
    metric, _ = silhouette_coeff(features=data.iloc[:, :-3].values,
                                              queries=(data.type == "CL").values, 
                                              labels=data.disease.to_numpy(),
                                              class_partition=(data.type == "CL").values,
                                              n_neighbours=100)
    tumour_cl_metrics.loc[model, "silhouette"] = np.mean(metric)

#### Mean Silhouette

In [ ]:
for model, data in tqdm(tumour_cl_z.items()):
    metric_disease = []
    for disease in selected_diseases:
        metric, _ = silhouette_coeff(features=data.iloc[:, :-3].values,
                                                  queries=(data.type == "CL").values, 
                                                  labels=data.disease.to_numpy(),
                                                  class_partition=(data.type == "CL").values,
                                                  n_neighbours=100)
        metric_disease.append(np.mean(metric))
    tumour_cl_metrics.loc[model, "mean-silhouette"] = np.mean(metric)

In [ ]:
tumour_cl_metrics

## 2.2 UCI Income Data

In [ ]:
uci_metrics = pd.DataFrame(columns=["kbet", "silhouette"], index=models)

#### kbet

In [ ]:
n_neighbours = 100
for model, data in tqdm(uci_z.items()):
    num_male = sum((data.type == "Male").values)
    num_female = sum((data.type != "Male").values)
    freq_female = n_neighbours * num_female / (num_male + num_female)
    freq_male = n_neighbours - freq_female

    _, counts = knn_metric(features=data.iloc[:, :-2].values,
                           queries=[True] * data.shape[0], # (data.type == "Male").values
                           labels=data.income.to_numpy(),
                           class_partition=(data.type == "Male").values,
                           n_neighbours=n_neighbours,
                           return_counts=True,
                           )
    expected_freq = np.where((data.type == "Male").values, freq_female, freq_male)
    uci_metrics.loc[model, "kbet"] = kbet(counts[:, 1], expected_freq=expected_freq, n_neighbours=n_neighbours, significance=0.01)

#### Silhouette

In [ ]:
for model, data in tqdm(uci_z.items()):
    metric, _ = silhouette_coeff(features=data.iloc[:, :-2].values,
                                  queries=(data.type == "Male").values, 
                                  labels=data.income.to_numpy(),
                                  class_partition=(data.type == "Male").values,
                                  n_neighbours=100)
    uci_metrics.loc[model, "silhouette"] = np.mean(metric)

In [ ]:
uci_metrics